## Preparation BEFORE the Loop:

- Designate a folder to save your information.
- Define custom functions you will use for your API calls
- Load your cleaned title basics data from Part 1 of Project 2 (or query your title_basics table in your MySQL database).
- Define the year you wish to retrieve (2010) and create an empty list for appending error messages.

In [1]:
# Import packages
import os, time, json
import tmdbsimple as tmdb 
import pandas as pd
from tqdm.notebook import tqdm_notebook
# Create the folder for saving files (if it doesn't exist)
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'Create Movies Database.sql',
 'final_tmdb_data_2010.csv.gz',
 'IMDB Movie Dataset Info.docx',
 'Project2 Model.mwb',
 'Screenshot 2023-09-28 232300.png',
 'title-akas-us-only.csv',
 'title.basics-cleaned.tsv.gz',
 'title.basics.tsv.gz',
 'title.ratings-cleaned.tsv.gz',
 'title.ratings.tsv.gz',
 'tmdb_api_results_2010.json']

In [2]:
import json
with open('/Users/thoma/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [3]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [4]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [5]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title.basics-cleaned.tsv.gz')

In [6]:
YEAR = 2010

In [7]:
errors = [ ]

## Prepare the DataFrame and JSON File

- Use the selected year to define filenames and filter the data

    - Define a JSON_FILE filename to save the results in progress.
    - Check if the file exists.
        - if it does not exist, create the empty JSON file with with open that just contains the key "imdb_id"
        - if it exists, print a message saying that it already exists.

In [8]:
#Defining the JSON file to store results for year
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

In [9]:
# Check if file exists
file_exists = os.path.isfile(JSON_FILE)
file_exists

True

In [10]:
# If it does not exist: create it
if file_exists == False:
    # Print a message indicating the file is being created 
    print(f"Creating {JSON_FILE} for API results for {YEAR}.")
    # save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)

In [11]:
file_exists = os.path.isfile(JSON_FILE)
file_exists

True

## Now that the JSON file for the results in progress exists:

- Filter the IMDB title basics data for the selected year and save the movie IDs from that year as "movies_ids".
- Check the JSON file for previously downloaded movie IDs and filter out the movie ids that already exists in the JSON file ( to prevent duplicate API calls) by:
    - Loading in the contents of the JSON file pd.read_json.
        - Compare the movie_ids that were in the JSON file to your saved movie_ids_to_get.
    - Save the final list of "movie_ids_to_get" by filtering out movies that already exists in the JSON file.

In [12]:
# Filtering for movies from selected startYear
df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
movie_ids = df['tconst']
movie_ids.head()

1138    tt0230212
3806    tt0312305
4258    tt0326965
4436    tt0331312
6670    tt0393049
Name: tconst, dtype: object

In [13]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)
previous_df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0312305,0.0,/lqUbt2cy2pnqvxKefbQAtxLS0WA.jpg,None,0.0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://www.qqthemovie.com/,23738.0,en,Quantum Quest: A Cassini Space Odyssey,...,0.0,45.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Quantum Quest: A Cassini Space Odyssey,0.0,7.900,8.0,
2,tt0326965,0.0,/xt2klJdKCVGXcoBGQrGfAS0aGDE.jpg,None,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",http://www.inmysleep.com,40048.0,en,In My Sleep,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sleepwalking can be deadly.,In My Sleep,0.0,5.318,33.0,PG-13
3,tt0331312,0.0,None,None,0.0,[],,214026.0,en,This Wretched Life,...,0.0,0.0,[],Released,,This Wretched Life,0.0,5.000,1.0,
4,tt0393049,0.0,/gc9FN5zohhzCt05RkejQIIPLtBl.jpg,None,300000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",,324352.0,en,Anderson's Cross,...,0.0,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sometimes the boy next door is more than the b...,Anderson's Cross,0.0,4.000,5.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2727,tt7851834,0.0,/c4x2XPngdiW8Vmc6EmmTXAgzViJ.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}]",,166919.0,ko,여의도,...,0.0,88.0,"[{'english_name': 'Korean', 'iso_639_1': 'ko',...",Released,,A Friend In Need,0.0,5.000,3.0,NaN
2728,tt8090084,0.0,None,None,0.0,"[{'id': 35, 'name': 'Comedy'}]",,516206.0,en,Goodbye Dolly,...,0.0,46.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Goodbye Dolly,0.0,0.000,0.0,
2729,tt8160720,0.0,None,None,0.0,[],,273502.0,en,Moist Fury,...,0.0,65.0,[],Released,,Moist Fury,0.0,6.000,2.0,
2730,tt9164254,1.0,None,None,0.0,[],,775304.0,en,"Text, Lies and Video",...,0.0,103.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Download Deception. Upload Revenge.,"Text, Lies and Video",0.0,0.000,0.0,NC-17


In [14]:
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

## Perform the Loop of API Calls
Note: you have already written a function to combine the certification with the rest of the .info() from the TMDB API results in the Intro to TMDB API lesson.

Create a loop to make API calls for each id in the YEAR specified. Include a progress bar using tqdm_notebook

**For each movie id:**

- Extract the current ID from the API and retrieve the dictionary of results
- Append the new results to the list from the JSON file
- Save the updated JSON file back to the disk

In [15]:
# Loop through movie_ids_to_get with a tqdm progress bar
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):

    # Attempt to retrieve then data for the movie id
    try:
        temp = get_movie_with_rating(movie_id)  #This uses your pre-ma    de function
        # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)

    # If it fails,  make a dict with just the id and None for certification.
    except Exception as e:
        errors.append([movie_id, e])

Movies from 2010:   0%|          | 0/1131 [00:00<?, ?it/s]

In [16]:
print(f"- Total errors: {len(errors)}")

- Total errors: 1131


## Save the Results to Compressed .csv
- After the loop, save the final results for the year as a csv.gz file with the year in the filename.

In [17]:
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)